In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kuma_llm_utils.llm import (
    OpenAIClient, OpenAIWorker, 
    AnthropicClient, AnthropicWorker,
    GoogleAIClient, GoogleAIWorker)
from kuma_llm_utils.pipeline import (LLMModule, LLMRouter, Compose)

In [3]:
import os

os.environ['ANTHROPIC_API_KEY'] = "ENTER_YOUR_API_KEY"
os.environ['OPENAI_API_KEY'] = "ENTER_YOUR_API_KEY"
os.environ['GOOGLE_AI_API_KEY']= "ENTER_YOUR_API_KEY"

# Load clients

In [4]:
openai_engine = OpenAIClient()
anthropic_engine = AnthropicClient()
google_engine = GoogleAIClient()

# Define the task

In [5]:
router_prompt = '''\
You task is to route the user to the correct engine for their query.
Choose your answer from 'default', 'anthropic', or 'google'.
Engine list:
'openai' - 'default'
'anthropic' - 'anthropic'
'gemini' - 'google'
Query:
{question}
'''

qa_prompt = '''\
What is your name?
'''

In [6]:
openai_router_worker = OpenAIWorker(
    engine=openai_engine,
    prompt_template=router_prompt,
    generation_params={'model': 'gpt-4o-mini'})
openai_qa_worker = OpenAIWorker(
    engine=openai_engine, 
    prompt_template=qa_prompt,
    generation_params={'model': 'gpt-4o-mini'})
anthropic_qa_worker = AnthropicWorker(
    engine=anthropic_engine, 
    prompt_template=qa_prompt,
    generation_params={'model': 'claude-3-5-haiku-latest', 'max_tokens': 1024})
gemini_qa_worker = GoogleAIWorker(
    engine=google_engine, 
    prompt_template=qa_prompt,
    generation_params={'model': 'gemini-2.0-flash-exp'})

In [7]:
pipeline = Compose([
    LLMRouter(
        route_llm=openai_router_worker,
        job_pipeline={
            'default': LLMModule(llm=openai_qa_worker, output_key='answer'),
            'anthropic': LLMModule(llm=anthropic_qa_worker, output_key='answer'),
            'google': LLMModule(llm=gemini_qa_worker, output_key='answer')
        }
    )
])

# Generate!

In [8]:
print(await pipeline.generate({'question': 'Use google models'}))

2025-01-28 18:41:12,480 - 77   - INFO     - OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 73}
2025-01-28 18:41:12,482 - 27   - INFO     - LLMRouter |  route = google
2025-01-28 18:41:14,725 - 70   - INFO     - GoogleAIClient |  gemini-2.0-flash-exp {'request': 1, 'input_token': 7, 'output_token': 12}
INFO:default:GoogleAIClient |  gemini-2.0-flash-exp {'request': 1, 'input_token': 7, 'output_token': 12}


{'question': 'Use google models', 'answer': 'I am a large language model, trained by Google.\n'}


In [9]:
print(await pipeline.generate({'question': 'Use anthropic models'}))

2025-01-28 18:41:15,296 - 77   - INFO     - OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 75}
INFO:default:OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 75}
2025-01-28 18:41:15,298 - 27   - INFO     - LLMRouter |  route = anthropic
INFO:default:LLMRouter |  route = anthropic
2025-01-28 18:41:16,349 - 83   - INFO     - AnthropicClient |  claude-3-5-haiku-latest {'request': 1, 'input_token': 12, 'output_token': 24}
INFO:default:AnthropicClient |  claude-3-5-haiku-latest {'request': 1, 'input_token': 12, 'output_token': 24}


{'question': 'Use anthropic models', 'answer': "I'm Claude, an AI created by Anthropic to be helpful, honest, and harmless."}


In [10]:
print(await pipeline.generate({'question': 'Use openai models'}))

2025-01-28 18:41:16,859 - 77   - INFO     - OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 74}
INFO:default:OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 74}
2025-01-28 18:41:16,861 - 27   - INFO     - LLMRouter |  route = default
INFO:default:LLMRouter |  route = default
2025-01-28 18:41:17,566 - 77   - INFO     - OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 25}
INFO:default:OpenAIClient |  gpt-4o-mini {'request': 1, 'token': 25}


{'question': 'Use openai models', 'answer': 'I am called Assistant! How can I help you today?'}
